In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123339")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FMQJFA43F to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-123339
Azure region: southcentralus
Subscription id: 8a56ebc5-caa7-4a43-b8ed-1c1895e46051
Resource group: aml-quickstarts-123339


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, "HDPPL", cluster_config)

compute_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"init_lr": uniform(0.0005, 0.005),"hidden_size": choice(0, 100, 120, 140, 180)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=5, slack_factor=None, slack_amount=0.2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = 'training', entry_script = 'train.py', compute_target = compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,hyperparameter_sampling=ps,policy=policy,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=25)

In [26]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a4e14a48-747b-4a3b-b90d-22585deb67e8
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_a4e14a48-747b-4a3b-b90d-22585deb67e8?wsid=/subscriptions/8a56ebc5-caa7-4a43-b8ed-1c1895e46051/resourcegroups/aml-quickstarts-123339/workspaces/quick-starts-ws-123339

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-28T10:03:31.137465][API][INFO]Experiment created<END>\n""<START>[2020-10-28T10:03:31.896208][GENERATOR][INFO]Trying to sample '25' jobs from the hyperparameter space<END>\n"<START>[2020-10-28T10:03:32.3295403Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-10-28T10:03:32.394643][GENERATOR][INFO]Successfully sampled '25' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_a4e14a48-747b-4a3b-b90d-22585deb67e8
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_a4e14a48-747b-4a3b-b90d-22585deb67e8?wsid=/subscriptions/8a5

{'runId': 'HD_a4e14a48-747b-4a3b-b90d-22585deb67e8',
 'target': 'HDPPL',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-28T10:03:30.804711Z',
 'endTimeUtc': '2020-10-28T10:14:33.521719Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/8a56ebc5-caa7-4a43-b8ed-1c1895e46051/resourceGroups/aml-quickstarts-123339/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-123339/experiments/**SCRUBBED**/runs/HD_a4e14a48-747b-4a3b-b90d-22585deb67e8_0, {\'error\': {\n    "additional_properties": {\n        "componentName": null\n    },\n    "error": {\n        "additional_properties": {\n            "severity": null\n        },\n        "code": "UserError",\n        "message": "User program failed with SystemExit: 2",\n        "details_uri": "https://aka.ms/azureml-known-errors",\n        "target": null

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
joblib.dump(best_run, 'model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###